<a href="https://colab.research.google.com/github/MAGalarza/Encriptado-Challenger/blob/master/Cumulos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
datos = pd.read_excel("CUMULO_CAUCION-20250131_Ori.xlsx")
datos.head()

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
import pandas as pd

# ... (código para cargar tus datos en un DataFrame llamado 'df') ...
nuevos_datos = datos.copy()

# 1. Convertir las columnas a tipo string (texto)
nuevos_datos['CUIT_TOMADOR'] = nuevos_datos['CUIT_TOMADOR'].astype(str)
nuevos_datos['NOMBRE_TOMADOR'] = nuevos_datos['NOMBRE_TOMADOR'].astype(str)
nuevos_datos['APELLIDO_TOMADOR'] = nuevos_datos['APELLIDO_TOMADOR'].astype(str)

# 2. Realizar la concatenación
nuevos_datos['NOMBRE_COMPLETO_TOMADOR'] = (nuevos_datos['CUIT_TOMADOR'].str.cat(nuevos_datos['NOMBRE_TOMADOR'], sep=' - ').str.cat(nuevos_datos['APELLIDO_TOMADOR'], sep=' '))

# 3. Mostrar las primeras filas para verificar
print(nuevos_datos[['CUIT_TOMADOR', 'NOMBRE_TOMADOR', 'APELLIDO_TOMADOR', 'NOMBRE_COMPLETO_TOMADOR']].head())

In [ ]:
# Hace el control del TC y actualiza el nuevo archivo

nuevos_datos['CONTROL_TC'] = nuevos_datos['IM_SUMA_ASEGURADA_MF'] / nuevos_datos['IM_SUMA_ASEGURADA_MO']
nuevos_datos

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['PAS_COMPLETO'] = nuevos_datos['ID_PRODUCTOR'].astype(str).str.cat(nuevos_datos['DE_PRODUCTOR'].astype(str), sep=' - ')

# 1. Convertir las columnas a tipo string (texto)
# nuevos_datos['ID_PRODUCTOR'] = nuevos_datos['ID_PRODUCTOR'].astype(str)
# nuevos_datos['DE_PRODUCTOR'] = nuevos_datos['DE_PRODUCTOR'].astype(str)


# 2. Realizar la concatenación
# nuevos_datos['PAS_COMPLETO'] = (nuevos_datos['ID_PRODUCTOR'].str.cat(nuevos_datos['DE_PRODUCTOR'], sep=' - '))

# 3. Mostrar las primeras filas para verificar
print(nuevos_datos[['ID_PRODUCTOR', 'DE_PRODUCTOR', 'PAS_COMPLETO']].head())

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos = nuevos_datos.assign(TIPO_EMISION=None)
nuevos_datos.loc[nuevos_datos['DESC_ENDOSO'].isnull(), 'TIPO_EMISION'] = 'Valor por defecto'
nuevos_datos.loc[nuevos_datos['DESC_ENDOSO'].notnull(), 'TIPO_EMISION'] = nuevos_datos['DESC_ENDOSO']

# Para verificar los cambios:
print(nuevos_datos[['DESC_ENDOSO', 'TIPO_EMISION']].head(10))

In [ ]:
nuevos_datos.head()

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos['MES'] = pd.to_datetime(nuevos_datos['FECHAEMI']).dt.month

In [ ]:
nuevos_datos.info()

In [ ]:
nuevos_datos.head()